In [6]:
from pandas import DataFrame, read_csv
from helpers.preprocess import preprocess_data
from helpers.preprocess import drop_unnecessary_columns

import pandas as pd 
import numpy as np

In [7]:
movies_md = r'dataset/movies_tmdbMeta.csv'
movies_md_df = pd.read_csv(movies_md)
pd.set_option('display.max_columns', 50)

In [8]:
min_vote_count = 1000

movies_md_df = preprocess_data(movies_md_df, min_vote_count)

für elise
['deafness', 'sister-in-law', 'character study', 'uncle nephew relationship', 'f\xc3\xbcr elise']
straßenkids
['black people', '1970s', 'jazz musician', 'stra\xc3\x9fenkids', 'dysfunctional family', 'independent film', 'teacher', 'urban', 'parenthood']
staatsanwältin
['staatsanw\xc3\xa4ltin', 'cops', 'gangster', 'stakeout', 'police stakeout']
fiancé
['father son relationship', 'lovesickness', 'journalist', 'loss of mother', 'loss of lover', 'radio station', 'christmas party', "love of one's life", 'radio', 'seattle', 'baltimore', 'fianc\xc3\xa9', 'radio presenter', 'airplane', 'radio transmission', 'observation deck', 'friendship', 'unsociability', 'woman director']
cinéma vérité
['politics', 'election campaign', 'woman director', 'cin\xc3\xa9ma v\xc3\xa9rit\xc3\xa9']
Øverste hylde
['skab under tv', '\xc3\x98verste hylde', 'foran']
Øverste hylde
['skab under tv', 'bagved', '\xc3\x98verste hylde']
protégé
['london england', 'morality', 'doctor', 'science', 'prot\xc3\xa9g\xc3\x

würzburg
['hunter', 'father son relationship', 'judge', 'drug traffic', 'pizza delivery', 'law', 'joint', 'chaos', 'undercover agent', 'investigation', 'w\xc3\xbcrzburg', 'marijuana', 'drug']
vichy-régime
['france', 'resistance', 'parent child relationship', 'vichy-r\xc3\xa9gime', 'german', 'deportation', 'jew persecution', 'auschwitz', 'jew', 'national socialism', 'childhood memory', 'biography', 'anti semitism', 'best friend', 'catholicism', 'resistance fighter']
daughter–in–law
['bridge', 'eavesdropping', 'son-in-law', 'family reunion', 'parents', 'elderly couple', 'retirement home', 'general store', 'dance band', 'septuagenarian', 'ballroom', 'daughter\xe2\x80\x93in\xe2\x80\x93law', 'rhumba']
Øverste hylde
['skab under tv', '\xc3\x98verste hylde', 'foran']
Øverste hylde
['skab under tv', '\xc3\x98verste hylde', 'foran']
fabergé egg
['sequel', 'faberg\xc3\xa9 egg', 'dutch eastindian company', 'second', 'part', 'golden egg', 'goon']
fiancé
['china', 'self sacrifice', 'fianc\xc3\xa9',

josé ignácio
['paris', 'london england', 'trip', 'buenos aires', 'sao paulo', 'porto', 'venezia', 'jos\xc3\xa9 ign\xc3\xa1cio', 'istambul']
jönsson gang
['stockholm', 'j\xc3\xb6nsson gang', 'city hall']
jönsson gang
['computer chip', 'stockholm', 'j\xc3\xb6nsson gang', 'ikea']
jönsson gang
['mallorca', 'cathedral', 'hot air balloon', 'safe', 'caper', 'beauty salon', 'burglary', 'j\xc3\xb6nsson gang', 'break in', 'caper comedy', 'criminal gang', 'palma de mallorca']
fiancé
['fianc\xc3\xa9', 'framed for murder']
cinéma français
['medicine', 'drama', 'dramedy', 'cin\xc3\xa9ma fran\xc3\xa7ais']
united states–mexico barrier
['illegal immigration', 'border patrol', 'united states\xe2\x80\x93mexico barrier', 'human trafficking', 'border crossing']
fiancé fiancée relationship
['african american', 'single mother', 'fianc\xc3\xa9 fianc\xc3\xa9e relationship', 'corporation', 'rich man - poor woman']
pokémon
['pok\xc3\xa9mon', 'anime']
любовь
['\xd0\xbb\xd1\x8e\xd0\xb1\xd0\xbe\xd0\xb2\xd1\x8c', '\

In [4]:
# GENRES

s = movies_md_df.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_df = movies_md_df.drop('genres', axis=1).join(s)
gen_df['genre'].value_counts().shape[0]

20

In [5]:
pop_gen = pd.DataFrame(gen_df['genre'].value_counts()).reset_index()
pop_gen.columns = ['genre', 'movies']
pop_gen.head(20)

,genre,movies
0,Drama,20808
1,Comedy,13467
2,Thriller,7793
3,Romance,6923
4,Action,6727
5,Horror,4760
6,Crime,4387
7,Documentary,4050
8,Adventure,3587
9,Science Fiction,3137


In [6]:
# Graphical Representation of Genres
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
sns.set(font_scale=1.25)

plt.figure(figsize=(18,8))
sns.barplot(x='genre', y='movies', data=pop_gen.head(20))
plt.show()

In [7]:
# PRODUCTION COUNTRIES

s = movies_md_df.apply(lambda x: pd.Series(x['production_countries']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'production_country'
gen_df = movies_md_df.drop('production_countries', axis=1).join(s)
gen_df['production_country'].value_counts().shape[0]

160

In [8]:
pop_gen = pd.DataFrame(gen_df['production_country'].value_counts()).reset_index()
pop_gen.columns = ['production_country', 'movies']
pop_gen.head(20)

,production_country,movies
0,United States of America,21484
1,United Kingdom,4186
2,France,4079
3,Germany,2324
4,Italy,2240
5,Canada,1795
6,Japan,1726
7,Spain,982
8,Russia,924
9,India,868


In [9]:
# Drop all the extra columns we don't need

movies_md_df = drop_unnecessary_columns(movies_md_df)
movies_md_df.head()

,belongs_to_collection,budget,homepage,popularity,runtime,spoken_languages,vote_average,keywords,cast_size,crew_size,production_score,is_english,rating,is_drama,is_comedy,is_thriller,is_action,is_romance,is_adventure,is_crime,is_science_fiction,is_horror,is_family,is_fantasy,is_mystery,is_animation,is_history,is_music,is_war,is_western,is_documentary,is_foreign,prod_uk,prod_usa,prod_france,prod_other
0,1,30000000.0,1,21.946943,81.0,1,7.7,"['jealousy', 'toy', 'boy', 'friendship', 'frie...",13,106,51.713007,1,3,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0
1,1,65000000.0,1,17.015539,104.0,2,6.9,"['board game', 'disappearance', ""based on chil...",26,16,47.511209,1,3,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0
2,1,NaN,1,11.712900,101.0,1,6.5,"['fishing', 'best friend', 'duringcreditssting...",7,4,3.483333,1,3,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1,16000000.0,1,3.859495,127.0,1,6.1,"['based on novel', 'interracial relationship',...",10,10,12.838144,1,2,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,1,NaN,1,8.387519,106.0,1,5.7,"['baby', 'midlife crisis', 'confidence', 'agin...",12,7,24.590805,1,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [9]:
# Make input and output files

y = movies_md_df[['rating']]
y.to_csv('dataset/Y.csv', index=False)
movies_md_df = movies_md_df.drop(['rating'], 1)

movies_md_df.to_csv('dataset/X.csv', index=True, index_label='Id')